In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
import random


import kerastuner as kt

def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

# From https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98

class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

import shutil
def del_his(n, v):
    try:
        shutil.rmtree(f'models_v{v}/jane_street_{n}')
        os.remove(f"models_v{v}/best_hp_{n}.pkl")
        # os.remove(f"models_v{v}/model_{n}_0.hdf5")
        os.remove(f"models_v{v}/model_{n}_0_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_1.hdf5")
        os.remove(f"models_v{v}/model_{n}_1_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_2.hdf5")
        os.remove(f"models_v{v}/model_{n}_2_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_3.hdf5")
        os.remove(f"models_v{v}/model_{n}_3_finetune.hdf5")
        os.remove(f"models_v{v}/model_{n}_4.hdf5")
        os.remove(f"models_v{v}/model_{n}_4_finetune.hdf5")
    except:
        pass

In [24]:
train = pd.read_csv('./train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train = train.query('weight > 0').reset_index(drop = True)
train.fillna(train.mean(),inplace=True)

features = [c for c in train.columns if 'feature' in c] + ["weight"]

from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(train[features].values)
y = train[['resp_1', 'resp_2', 'resp_3', 'resp_4']].values

f_mean = np.mean(train[features[1:]].values,axis=0)

In [30]:
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4']
X_resp = train[['resp_1', 'resp_2', 'resp_3', 'resp_4']].values
y_resp = np.stack([(train['resp'] > 0.000001).astype('int')]).T
dates = train['date'].to_numpy()

In [40]:
import pickle
[features, resp_cols, f_mean, x_scaler, y_scaler, models_resp] = pickle.load(open(f"models_v{v}/conf.pkl", "rb"))

In [41]:
## v7

def create_autoencoder(input_dim,output_dim,noise=0.05):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(640,activation='relu')(encoded)
    decoded = Dropout(0.2)(encoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    x = Dense(320,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=encoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.001),loss={'decoded':'mse','label_output':'mse'})
    return autoencoder, encoder

def create_model(hp,input_dim,output_dim,encoder):
    inputs = Input(input_dim)
    
    x = encoder(inputs)
    x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(x)
    x = Dropout(hp.Float('init_dropout',0.0,0.5))(x)
    
    for i in range(hp.Int('num_layers',1,5)):
        x = Dense(hp.Int(f'num_units_{i}',128,256))(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(hp.Float(f'dropout_{i}',0.0,0.5))(x)
    x = Dense(output_dim,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.1,default=0.001)),loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing',0.0,0.1)),metrics="mse")
    return model

v = 7
try:
    os.mkdir(f"models_v{v}")
except:
    pass

In [6]:
import pickle
pickle.dump([features, resp_cols, f_mean, scaler], open(f"models_v{v}/conf.pkl", "wb"))

In [36]:
del_his(5, v)

In [32]:
%%time
SEEDS = [5, 10, 15, 20]
FOLDS = 5

for j, SEED in enumerate(SEEDS):
    set_all_seeds(SEED)

    autoencoder, encoder = create_autoencoder(X.shape[-1],len(resp_cols),noise=0.1)
    autoencoder.fit(X,(X,y),
                    epochs=1000,
                    batch_size=4096*4, 
                    validation_split=0.1,
                    callbacks=[EarlyStopping('val_loss',patience=10,restore_best_weights=True)])

    encoder.trainable = False
    model_fn = lambda hp: create_model(hp,X.shape[-1],len(resp_cols), encoder)

    tuner = CVTuner(
        hypermodel=model_fn,
        oracle=kt.oracles.BayesianOptimization(
        objective= kt.Objective('val_mse', direction='min'),
        num_initial_points=4,
        max_trials=20,
        seed=SEED),
        project_name=f'models_v{v}/jane_street_{SEED}'
        )

    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
    splits = list(gkf.split(y, groups=train['date'].values))
    tuner.search((X,),(y,),splits=splits,batch_size=4096*2,epochs=300,callbacks=[EarlyStopping('val_mse', mode='max',patience=5)])
    hp  = tuner.get_best_hyperparameters(1)[0]
    oof = np.zeros(y.shape)
    pd.to_pickle(hp,f'models_v{v}/best_hp_{SEED}.pkl')
    for fold, (train_indices, test_indices) in enumerate(splits):
        model = model_fn(hp)
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=300,batch_size=4096*2,callbacks=[EarlyStopping('val_mse',patience=10,restore_best_weights=True)])

        # model.save_weights(f'./model_{SEED}_{fold}.hdf5')
        model.compile(Adam(hp.get('lr')/100),loss='mse')
        model.fit(X_test,y_test,epochs=6, batch_size=16384)
        # encoder.save_weights(f'models_v{v}/encoder_{SEED}_{fold}.hdf5')
        model.save_weights(f'models_v{v}/model_{SEED}_{fold}_finetune.hdf5')

Epoch 1/1000
87/87 [==============================] - 4s 45ms/step - loss: 0.1392 - decoded_loss: 0.0406 - label_output_loss: 0.0986 - val_loss: 0.0657 - val_decoded_loss: 0.0493 - val_label_output_loss: 0.0164
Epoch 2/1000
87/87 [==============================] - 3s 40ms/step - loss: 0.0184 - decoded_loss: 0.0088 - label_output_loss: 0.0097 - val_loss: 0.0297 - val_decoded_loss: 0.0282 - val_label_output_loss: 0.0015
Epoch 3/1000
87/87 [==============================] - 4s 41ms/step - loss: 0.0062 - decoded_loss: 0.0044 - label_output_loss: 0.0018 - val_loss: 0.0117 - val_decoded_loss: 0.0110 - val_label_output_loss: 6.5261e-04
Epoch 4/1000
87/87 [==============================] - 3s 40ms/step - loss: 0.0035 - decoded_loss: 0.0027 - label_output_loss: 8.3244e-04 - val_loss: 0.0054 - val_decoded_loss: 0.0048 - val_label_output_loss: 5.7731e-04
Epoch 5/1000
87/87 [==============================] - 3s 40ms/step - loss: 0.0026 - decoded_loss: 0.0019 - label_output_loss: 6.3360e-04 - val_l

87/87 [==============================] - 4s 41ms/step - loss: 7.4375e-04 - decoded_loss: 3.2007e-04 - label_output_loss: 4.2368e-04 - val_loss: 5.2078e-04 - val_decoded_loss: 6.7842e-05 - val_label_output_loss: 4.5294e-04
Epoch 37/1000
87/87 [==============================] - 4s 41ms/step - loss: 7.3518e-04 - decoded_loss: 3.1206e-04 - label_output_loss: 4.2312e-04 - val_loss: 5.1708e-04 - val_decoded_loss: 6.7225e-05 - val_label_output_loss: 4.4985e-04
Epoch 38/1000
87/87 [==============================] - 3s 40ms/step - loss: 7.2637e-04 - decoded_loss: 3.0344e-04 - label_output_loss: 4.2293e-04 - val_loss: 5.1306e-04 - val_decoded_loss: 6.3335e-05 - val_label_output_loss: 4.4973e-04
Epoch 39/1000
87/87 [==============================] - 4s 40ms/step - loss: 7.1827e-04 - decoded_loss: 2.9548e-04 - label_output_loss: 4.2279e-04 - val_loss: 5.1324e-04 - val_decoded_loss: 6.3654e-05 - val_label_output_loss: 4.4958e-04
Epoch 40/1000
87/87 [==============================] - 4s 41ms/step - 

Epoch 71/1000
87/87 [==============================] - 4s 42ms/step - loss: 5.8593e-04 - decoded_loss: 1.6500e-04 - label_output_loss: 4.2093e-04 - val_loss: 4.8560e-04 - val_decoded_loss: 3.6180e-05 - val_label_output_loss: 4.4942e-04
Epoch 72/1000
87/87 [==============================] - 4s 41ms/step - loss: 5.8480e-04 - decoded_loss: 1.6377e-04 - label_output_loss: 4.2103e-04 - val_loss: 4.8705e-04 - val_decoded_loss: 3.6490e-05 - val_label_output_loss: 4.5056e-04
Epoch 73/1000
87/87 [==============================] - 4s 42ms/step - loss: 5.8463e-04 - decoded_loss: 1.6365e-04 - label_output_loss: 4.2098e-04 - val_loss: 4.8796e-04 - val_decoded_loss: 3.7735e-05 - val_label_output_loss: 4.5022e-04
Epoch 74/1000
87/87 [==============================] - 4s 41ms/step - loss: 5.8393e-04 - decoded_loss: 1.6304e-04 - label_output_loss: 4.2088e-04 - val_loss: 5.3345e-04 - val_decoded_loss: 3.8942e-05 - val_label_output_loss: 4.9451e-04
Epoch 75/1000
87/87 [==============================] - 4

Epoch 106/1000
87/87 [==============================] - 4s 41ms/step - loss: 5.6893e-04 - decoded_loss: 1.4804e-04 - label_output_loss: 4.2089e-04 - val_loss: 4.7968e-04 - val_decoded_loss: 2.9957e-05 - val_label_output_loss: 4.4972e-04
Epoch 107/1000
87/87 [==============================] - 4s 42ms/step - loss: 5.7027e-04 - decoded_loss: 1.4921e-04 - label_output_loss: 4.2106e-04 - val_loss: 0.0188 - val_decoded_loss: 3.2148e-05 - val_label_output_loss: 0.0188
Epoch 108/1000
87/87 [==============================] - 4s 41ms/step - loss: 5.6832e-04 - decoded_loss: 1.4672e-04 - label_output_loss: 4.2160e-04 - val_loss: 0.0010 - val_decoded_loss: 2.9932e-05 - val_label_output_loss: 0.0010
Epoch 109/1000
87/87 [==============================] - 4s 41ms/step - loss: 5.6716e-04 - decoded_loss: 1.4571e-04 - label_output_loss: 4.2146e-04 - val_loss: 5.4337e-04 - val_decoded_loss: 3.0675e-05 - val_label_output_loss: 5.1269e-04
Epoch 110/1000
87/87 [==============================] - 4s 42ms/step

18/18 [==============================] - 1s 44ms/step - loss: 0.0640 - mse: 0.0026 - val_loss: 0.1816 - val_mse: 4.5053e-04
Epoch 20/300
18/18 [==============================] - 1s 42ms/step - loss: -0.0783 - mse: 0.0019 - val_loss: 0.1498 - val_mse: 6.6030e-04
Epoch 21/300
18/18 [==============================] - 1s 42ms/step - loss: -0.3809 - mse: 0.0026 - val_loss: 0.1597 - val_mse: 6.5517e-04
Epoch 22/300
18/18 [==============================] - 1s 42ms/step - loss: -0.8486 - mse: 0.0097 - val_loss: 0.6766 - val_mse: 4.3323e-04
Epoch 23/300
18/18 [==============================] - 1s 42ms/step - loss: -1.6561 - mse: 0.0061 - val_loss: 3.0672 - val_mse: 0.4715
Epoch 24/300
18/18 [==============================] - 1s 42ms/step - loss: -3.0811 - mse: 0.0072 - val_loss: 6.9842 - val_mse: 0.5463
Epoch 25/300
18/18 [==============================] - 1s 41ms/step - loss: -4.2153 - mse: 0.0224 - val_loss: 0.1647 - val_mse: 0.0823
Epoch 26/300
18/18 [==============================] - 1s 45m

KeyboardInterrupt: 

In [ ]:
if test_df['weight'].item() > 0:
    x_tt = test_df.loc[:, features].values
    if np.isnan(x_tt[:, 1:].sum()):
        x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
    x_tt = x_scaler.transform(x_tt)
    pred = np.mean([model.predict(x_tt) for model in models],axis=0)
#         pred = np.mean(pred)
    pred = np.mean([model.predict(pred) for model in models_resp])
    pred_df.action = np.where(pred >= th, 1, 0).astype(int)
else:
    pred_df.action = 0
env.predict(pred_df)